# One Shot

Ziel ist es, mit einem Request alle Antworten zu erhalten.

In [ ]:
sys.path.append('..')

import sys
import time
import survey.models 
from tqdm.notebook import tqdm
from survey.request import Request


# strategie laden
strategies = survey.models.StrategyModel()
strategies.load_strategies('oneshot_%')
strategies_list = strategies.get_strategies()

# fragen laden
questions = survey.models.QuestionsModel()
questions_list = questions.get_questions()

# models laden
models = survey.models.ModelsModel()
models_list = models.get_models()

# request und response laden
run = survey.models.RunModel()
requests = Request()
response = survey.models.ResponseModel()

In [ ]:
print(f"Strategies: {len(strategies_list)} - {strategies_list}")
print(f"Questions: {len(questions_list)}")
print(f"Models: {len(models_list)}")

In [ ]:
for strategy in tqdm(strategies_list, desc="Strategies"):

    print(f"\n  Strategy: {strategy['name']}")

    # prompts lesen
    with open(f"../{strategy['system_path']}", "r") as f:
        system_prompt = f.read()

    with open(f"../{strategy['message_path']}", "r") as f:
        message_prompt = f.read()

    # questions
    questions_text = "\n".join([f"{id}. {text}" for id, text in questions_list])
    message = message_prompt.replace("{questions}", questions_text)


    for model_db_id, model_id in tqdm(models_list, desc=f"  Models ({strategy['name']})"):

        print(f"    {model_id}", end=" ", flush=True)

        # Run sichern
        run_id = run.write_run(model_db_id, strategies_list['id'])

        # Befragung
        start_time = time.time()

        answer = requests.request_model(model_id, system_prompt, message)

        duration_time = time.time() - start_time

        # DEBUG - Was kommt zurück?
        print(f"\n=== Model: {model_id} ===")
        print(answer.choices[0].message.content)
        print("=" * 50)

        try:
            response.write_respone(model_db_id, duration_time, answer, 'COMPLETE')
            run.update_run(answer.usage.prompt_tokens, answer.usage.completion_tokens, duration_time, run_id)
        except Exception as e:
            print(f"    --- Error ---")
